# Creating annomaly plots with interannual variability between months for seasonal changes in phytoplankton biomass! 
### INCLUDING  ice fraction

In [1]:
import warnings
warnings.simplefilter("ignore") # Silence warnings
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import numpy as np
from matplotlib.pyplot import figure

import numpy.ma as ma
from netCDF4 import Dataset as NetCDFFile
import cartopy
import cartopy.crs as ccrs
import pylab 
from copy import deepcopy
import pandas as pd
import matplotlib
import matplotlib.colors as colors
from cartopy.util import add_cyclic_point
plt.rcParams['mathtext.default']='regular'
from collections import OrderedDict
import cmocean
import matplotlib.cm as cm
import matplotlib as mpl
import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib
# Scientific libraries
from numpy import arange,array,ones
from scipy import stats
import os

%matplotlib inline
from glob import glob
import dask
import esmlab
import pop_tools 
from statistics import mean

In [2]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        if clip is None:
            clip = self.clip

        result, is_scalar = self.process_value(value)

        self.autoscale_None(result)
        vmin, vmax, midpoint = self.vmin, self.vmax, self.midpoint

        if not (vmin < midpoint < vmax):
            raise ValueError("midpoint must be between maxvalue and minvalue.")
        elif vmin == vmax:
            result.fill(0) # Or should it be all masked? Or 0.5?
        elif vmin > vmax:
            raise ValueError("maxvalue must be bigger than minvalue")
        else:
            vmin = float(vmin)
            vmax = float(vmax)
            if clip:
                mask = np.ma.getmask(result)
                result = np.ma.array(np.clip(result.filled(vmax), vmin, vmax),
                                  mask=mask)

            # ma division is very slow; we can take a shortcut
            resdat = result.data

            #First scale to -1 to 1 range, than to from 0 to 1.
            resdat -= midpoint
            resdat[resdat>0] /= abs(vmax - midpoint)
            resdat[resdat<0] /= abs(vmin - midpoint)

            resdat /= 2.
            resdat += 0.5
            result = np.ma.array(resdat, mask=result.mask, copy=False)

        if is_scalar:
            result = result[0]
        return result

In [3]:
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [4]:
def adjust_pop_grid(tlon,tlat,field):
    nj = tlon.shape[0]
    ni = tlon.shape[1]
    xL = int(ni/2 - 1)
    xR = int(xL + ni)

    tlon = np.where(np.greater_equal(tlon,np.min(tlon[:,0])),tlon-360.,tlon)
    lon  = np.concatenate((tlon,tlon+360.),1)
    lon = lon[:,xL:xR]

    if ni == 320:
        lon[367:-3,0] = lon[367:-3,0]+360.
    lon = lon - 360.
    lon = np.hstack((lon,lon[:,0:1]+360.))
    if ni == 320:
        lon[367:,-1] = lon[367:,-1] - 360.

    #-- trick cartopy into doing the right thing:
    #   it gets confused when the cyclic coords are identical
    lon[:,0] = lon[:,0]-1e-8

    #-- periodicity
    lat  = np.concatenate((tlat,tlat),1)
    lat = lat[:,xL:xR]
    lat = np.hstack((lat,lat[:,0:1]))

    field = np.ma.concatenate((field,field),1)
    field = field[:,xL:xR]
    field = np.ma.hstack((field,field[:,0:1]))
    return lon,lat,field

## Seasonality and interannual variability in SPECTRA

In [5]:
case = 'g.e21.G1850ECOIAF.t62_g17.marbl0_33.GNG595'
path = f'/glade/campaign/cesm/development/bgcwg/projects/marbl-spectra/{case}/ocn/hist'

In [6]:
variables = [f'{var}' for var in ['diat1C','diat2C','diat3C', 'ppC', 'diazC', 'mp1C','mp2C','mp3C','mp4C','ECOSYS_IFRAC','TEMP','NO3','photoC_TOT']]
coords = {'x':'TLONG','y':'TLAT'}
keep_vars = variables + list(coords.values())+['dz','KMT','time']

In [7]:
%%time

ds_summer_avg = xr.Dataset()
ds_spring_avg = xr.Dataset()
ds_fall_avg = xr.Dataset()
ds_winter_avg =xr.Dataset()

for year in np.arange(63,125,1):
    yr4="0{:02d}".format(year).zfill(4)
    print(year)
    
    ds_summer = xr.Dataset()
    ds_spring = xr.Dataset()
    ds_winter = xr.Dataset()
    ds_fall = xr.Dataset()

    file = sorted(glob(f'{path}/{case}.pop.h.{yr4}-*.nc'))
    
    winter_indexes = [0,1,-1]
    winter_files = [file[ind] for ind in winter_indexes]
    dsv_winter=xr.open_mfdataset(winter_files, decode_times=True,drop_variables=["transport_components", "transport_regions"], 
                            parallel=True, compat="override", combine='nested', concat_dim="time",data_vars="minimal",coords='minimal' )
        
    dsv_summer=xr.open_mfdataset(file[5:8], decode_times=True,drop_variables=["transport_components", "transport_regions"], 
                            parallel=True, compat="override", combine='nested', concat_dim="time",data_vars="minimal",coords='minimal' )
    dsv_spring=xr.open_mfdataset(file[2:5], decode_times=True,drop_variables=["transport_components", "transport_regions"], 
                            parallel=True, compat="override", combine='nested', concat_dim="time",data_vars="minimal",coords='minimal' )
    dsv_fall=xr.open_mfdataset(file[8:11], decode_times=True,drop_variables=["transport_components", "transport_regions"], 
                            parallel=True, compat="override", combine='nested', concat_dim="time",data_vars="minimal",coords='minimal' )
    
    for vv in variables: 
        ds_summer = xr.merge((ds_summer, dsv_summer[vv]))
        ds_spring = xr.merge((ds_spring, dsv_spring[vv]))
        ds_winter = xr.merge((ds_winter, dsv_winter[vv]))
        ds_fall = xr.merge((ds_fall, dsv_fall[vv]))
        

    ds_summer = ds_summer.drop([v for v in ds_summer.variables if v not in keep_vars]).squeeze()
    ds_summer = ds_summer.mean(dim='time')
    ds_summer_avg = xr.concat([ds_summer_avg, ds_summer],dim='year')
    
    ds_spring = ds_spring.drop([v for v in ds_spring.variables if v not in keep_vars]).squeeze()
    ds_spring = ds_spring.mean(dim='time')
    ds_spring_avg = xr.concat([ds_spring_avg, ds_spring],dim='year')
    
    ds_winter = ds_winter.drop([v for v in ds_winter.variables if v not in keep_vars]).squeeze()
    ds_winter = ds_winter.mean(dim='time')
    ds_winter_avg = xr.concat([ds_winter_avg, ds_winter],dim='year')
    
    ds_fall = ds_fall.drop([v for v in ds_fall.variables if v not in keep_vars]).squeeze()
    ds_fall = ds_fall.mean(dim='time')
    ds_fall_avg = xr.concat([ds_fall_avg, ds_fall],dim='year')

63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
CPU times: user 13min 6s, sys: 6min 29s, total: 19min 36s
Wall time: 20min 35s


In [8]:
ds_summer_avg.photoC_TOT

,Array,Chunk
Bytes,435.94 MiB,7.03 MiB
Shape,"(62, 15, 384, 320)","(1, 15, 384, 320)"
Count,2820 Tasks,62 Chunks
Type,float32,numpy.ndarray


In [13]:
time = np.arange(0,62,1)

In [14]:
phyto_bmss_summer = xr.DataArray(ds_summer_avg.photoC_TOT, coords={'time':time,'z_t_150m':dsv_fall.z_t_150m, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['time','z_t_150m', 'nlat', 'nlon'])
phyto_bmss_summer = phyto_bmss_summer.to_dataset(name='photoC_TOT_summer')

In [17]:
phyto_bmss_summer.to_netcdf('/glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_summer.nc');

	NC4_create: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_summer.nc cmode 0x1000 parameters (nil)
	HDF5 error messages turned on.
			nc4_create_file: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_summer.nc mode 0x1000
			nc4_grp_list_add: name / 
		nc_inq_format: ncid 0x400000
		NC4_inq_format_extended: ncid 0x400000
		nc_inq_typeids: ncid 0x400000
		NC4_inq: ncid 0x400000
		NC4_inq: ncid 0x400000
		nc_inq_grps: ncid 0x400000
		NC4_def_dim: ncid 0x400000 name time len 62
		NC4_def_dim: ncid 0x400000 name z_t_150m len 15
		NC4_def_dim: ncid 0x400000 name nlat len 384
		NC4_def_dim: ncid 0x400000 name nlon len 320
		NC4_def_var: name time type 10 ndims 1
		NC4_inq_unlimdims: ncid 0x400000
		NC4_inq_var_all: ncid 0x400000 varid 0
		NC4_inq_var_all: ncid 0x400000 varid 0
		NC4_inq_var_all: ncid 0x400000 varid 0
		NC4_inq_var_all: ncid 0x400000 varid 0
		NC4_inq_dim: ncid 0x400000 dimid 0
		NC4_inq_dim: ncid 0x400000 dimid 0
		NC4_inq_var_all: ncid 0

In [16]:
%%time
phyto_bmss_spring = xr.DataArray(ds_spring_avg.photoC_TOT, coords={'time':time,'z_t_150m':dsv_fall.z_t_150m, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['time','z_t_150m', 'nlat', 'nlon'])
phyto_bmss_spring = phyto_bmss_spring.to_dataset(name='photoC_TOT_spring')

CPU times: user 6.13 s, sys: 11 s, total: 17.1 s
Wall time: 48.8 s


In [18]:
phyto_bmss_spring.to_netcdf('/glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_spring.nc');

C4_inq_var_all: ncid 0x400000 varid 0
		NC4_inq_var_all: ncid 0x400000 varid 0
		NC4_inq_dim: ncid 0x400000 dimid 0
		NC4_inq_var_all: ncid 0x400000 varid 0
			NC4_put_vars: var->hdr.name time mem_nc_type 10
			nc4_enddef_netcdf4_file
			sync_netcdf4_file
		*** NetCDF-4 Internal Metadata: int_ncid 0x400000 ext_ncid 0x400000
		FILE - path: /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_summer.nc cmode: 0x1009 parallel: 0 redef: 0 fill_mode: 0 no_write: 0 next_nc_grpid: 1
		 GROUP - / nc_grpid: 0 nvars: 1 natts: 0
		 DIMENSION - dimid: 0 name: time len: 62 unlimited: 0
		 DIMENSION - dimid: 1 name: z_t_150m len: 15 unlimited: 0
		 DIMENSION - dimid: 2 name: nlat len: 384 unlimited: 0
		 DIMENSION - dimid: 3 name: nlon len: 320 unlimited: 0
		 VARIABLE - varid: 0 name: time ndims: 1 dimscale: 1 dimids: 0
			nc4_rec_write_groups_types: grp->hdr.name /
			nc4_rec_write_metadata: grp->hdr.name /, bad_coord_order 1
			var_create_dataset:: name time
	NC4_redef: ncid 0x400000
		NC

In [19]:
%%time
phyto_bmss_fall = xr.DataArray(ds_fall_avg.photoC_TOT, coords={'time':time,'z_t_150m':dsv_fall.z_t_150m, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['time','z_t_150m', 'nlat', 'nlon'])
phyto_bmss_fall = phyto_bmss_fall.to_dataset(name='photoC_TOT_fall')

CPU times: user 5.37 s, sys: 10.7 s, total: 16.1 s
Wall time: 47.2 s


In [20]:
phyto_bmss_fall.to_netcdf('/glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_fall.nc');

	NC4_create: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_fall.nc cmode 0x1000 parameters (nil)
	HDF5 error messages turned on.
			nc4_create_file: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_fall.nc mode 0x1000
			nc4_grp_list_add: name / 
		nc_inq_format: ncid 0x80000
		NC4_inq_format_extended: ncid 0x80000
		nc_inq_typeids: ncid 0x80000
		NC4_inq: ncid 0x80000
		NC4_inq: ncid 0x80000
		nc_inq_grps: ncid 0x80000
		NC4_def_dim: ncid 0x80000 name time len 62
		NC4_def_dim: ncid 0x80000 name z_t_150m len 15
		NC4_def_dim: ncid 0x80000 name nlat len 384
		NC4_def_dim: ncid 0x80000 name nlon len 320
		NC4_def_var: name time type 10 ndims 1
		NC4_inq_unlimdims: ncid 0x80000
		NC4_inq_var_all: ncid 0x80000 varid 0
		NC4_inq_var_all: ncid 0x80000 varid 0
		NC4_inq_var_all: ncid 0x80000 varid 0
		NC4_inq_var_all: ncid 0x80000 varid 0
		NC4_inq_dim: ncid 0x80000 dimid 0
		NC4_inq_dim: ncid 0x80000 dimid 0
		NC4_inq_var_all: ncid 0x80000 varid 0
		NC4_

In [21]:
%%time
phyto_bmss_winter = xr.DataArray(ds_winter_avg.photoC_TOT, coords={'time':time,'z_t_150m':dsv_fall.z_t_150m, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['time','z_t_150m', 'nlat', 'nlon'])
phyto_bmss_winter = phyto_bmss_winter.to_dataset(name='photoC_TOT_winter')

nits file_type 2 mem_type 2 len 11
			adding attribute units to the list...
			nc4_att_list_add: name units 
			nc4_convert_type: len 11 src_type 2 dest_type 2
		nc_inq_format: ncid 0x80000
	nc4_put_att: ncid 0x80000 varid 1 name positive file_type 2 mem_type 2 len 4
	nc4_put_att: ncid 0x80000 varid 1 name positive file_type 2 mem_type 2 len 4
			adding attribute positive to the list...
			nc4_att_list_add: name positive 
			nc4_convert_type: len 4 src_type 2 dest_type 2
		nc_inq_format: ncid 0x80000
	nc4_put_att: ncid 0x80000 varid 1 name valid_min file_type 5 mem_type 5 len 1
	nc4_put_att: ncid 0x80000 varid 1 name valid_min file_type 5 mem_type 5 len 1
			adding attribute valid_min to the list...
			nc4_att_list_add: name valid_min 
			nc4_convert_type: len 1 src_type 5 dest_type 5
		nc_inq_format: ncid 0x80000
	nc4_put_att: ncid 0x80000 varid 1 name valid_max file_type 5 mem_type 5 len 1
	nc4_put_att: ncid 0x80000 varid 1 name valid_max file_type 5 mem_type 5 len 1
			adding attrib

CPU times: user 6.18 s, sys: 10.4 s, total: 16.6 s
Wall time: 52.4 s


In [22]:
phyto_bmss_winter.to_netcdf('/glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_winter.nc');

	NC4_create: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_winter.nc cmode 0x1000 parameters (nil)
	HDF5 error messages turned on.
			nc4_create_file: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_winter.nc mode 0x1000
			nc4_grp_list_add: name / 
		nc_inq_format: ncid 0x490000
		NC4_inq_format_extended: ncid 0x490000
		nc_inq_typeids: ncid 0x490000
		NC4_inq: ncid 0x490000
		NC4_inq: ncid 0x490000
		nc_inq_grps: ncid 0x490000
		NC4_def_dim: ncid 0x490000 name time len 62
		NC4_def_dim: ncid 0x490000 name z_t_150m len 15
		NC4_def_dim: ncid 0x490000 name nlat len 384
		NC4_def_dim: ncid 0x490000 name nlon len 320
		NC4_def_var: name time type 10 ndims 1
		NC4_inq_unlimdims: ncid 0x490000
		NC4_inq_var_all: ncid 0x490000 varid 0
		NC4_inq_var_all: ncid 0x490000 varid 0
		NC4_inq_var_all: ncid 0x490000 varid 0
		NC4_inq_var_all: ncid 0x490000 varid 0
		NC4_inq_dim: ncid 0x490000 dimid 0
		NC4_inq_dim: ncid 0x490000 dimid 0
		NC4_inq_var_all: ncid 0

In [8]:
%%time
phytoC_summer = np.stack([ds_summer_avg.ppC, ds_summer_avg.mp1C, ds_summer_avg.diazC, 
                   ds_summer_avg.mp2C, ds_summer_avg.diat1C, ds_summer_avg.diat2C, 
                   ds_summer_avg.mp3C, ds_summer_avg.diat3C, ds_summer_avg.mp4C])

CPU times: user 57.8 s, sys: 3min 12s, total: 4min 10s
Wall time: 9min 45s


In [9]:
%%time
phytoC_spring = np.stack([ds_spring_avg.ppC, ds_spring_avg.mp1C, ds_spring_avg.diazC, 
                   ds_spring_avg.mp2C, ds_spring_avg.diat1C, ds_spring_avg.diat2C, 
                   ds_spring_avg.mp3C, ds_spring_avg.diat3C, ds_spring_avg.mp4C])

CPU times: user 58.9 s, sys: 3min 12s, total: 4min 11s
Wall time: 9min 42s


In [10]:
%%time
phytoC_winter = np.stack([ds_winter_avg.ppC, ds_winter_avg.mp1C, ds_winter_avg.diazC, 
                   ds_winter_avg.mp2C, ds_winter_avg.diat1C, ds_winter_avg.diat2C, 
                   ds_winter_avg.mp3C, ds_winter_avg.diat3C, ds_winter_avg.mp4C])

CPU times: user 59 s, sys: 3min 14s, total: 4min 13s
Wall time: 10min 14s


In [11]:
%%time
phytoC_fall = np.stack([ds_fall_avg.ppC, ds_fall_avg.mp1C, ds_fall_avg.diazC, 
                   ds_fall_avg.mp2C, ds_fall_avg.diat1C, ds_fall_avg.diat2C, 
                   ds_fall_avg.mp3C, ds_fall_avg.diat3C, ds_fall_avg.mp4C])

CPU times: user 58.8 s, sys: 3min 13s, total: 4min 12s
Wall time: 9min 38s


### Biomass

In [29]:
phyto_bmss_depth = xr.DataArray(np.nansum(phytoC_summer,axis=0), coords={'year':ds_fall_avg.year,'z_t_150m':dsv_fall.z_t_150m, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['year','z_t_150m', 'nlat', 'nlon'])
phyto_bmss_depth = phyto_bmss_depth.to_dataset(name='phytoC_summer')

phyto_bmss_depth['phytoC_spring'] = xr.DataArray(np.nansum(phytoC_spring,axis=0), coords={'year':ds_fall_avg.year,'z_t_150m':dsv_fall.z_t_150m, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['year','z_t_150m', 'nlat', 'nlon'])

phyto_bmss_depth['phytoC_fall'] = xr.DataArray(np.nansum(phytoC_fall,axis=0), coords={'year':ds_fall_avg.year,'z_t_150m':dsv_fall.z_t_150m, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['year','z_t_150m', 'nlat', 'nlon'])

phyto_bmss_depth['phytoC_winter'] = xr.DataArray(np.nansum(phytoC_winter,axis=0), coords={'year':ds_fall_avg.year,'z_t_150m':dsv_fall.z_t_150m, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['year','z_t_150m', 'nlat', 'nlon'])

In [30]:
phyto_bmss_depth.to_netcdf('/glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_depth.nc');

	NC4_create: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_depth.nc cmode 0x1000 parameters (nil)
	HDF5 error messages turned on.
			nc4_create_file: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr_depth.nc mode 0x1000
			nc4_grp_list_add: name / 
		nc_inq_format: ncid 0x5c0000
		NC4_inq_format_extended: ncid 0x5c0000
		nc_inq_typeids: ncid 0x5c0000
		NC4_inq: ncid 0x5c0000
		NC4_inq: ncid 0x5c0000
		nc_inq_grps: ncid 0x5c0000
		NC4_def_dim: ncid 0x5c0000 name year len 62
		NC4_def_dim: ncid 0x5c0000 name z_t_150m len 15
		NC4_def_dim: ncid 0x5c0000 name nlat len 384
		NC4_def_dim: ncid 0x5c0000 name nlon len 320
		NC4_def_var: name year type 10 ndims 1
		NC4_inq_unlimdims: ncid 0x5c0000
		NC4_inq_var_all: ncid 0x5c0000 varid 0
		NC4_inq_var_all: ncid 0x5c0000 varid 0
		NC4_inq_var_all: ncid 0x5c0000 varid 0
		NC4_inq_var_all: ncid 0x5c0000 varid 0
		NC4_inq_dim: ncid 0x5c0000 dimid 0
		NC4_inq_dim: ncid 0x5c0000 dimid 0
		NC4_inq_var_all: ncid 0x5

In [31]:
phyto_bmss_depth

<xarray.Dataset>
Dimensions:        (year: 62, z_t_150m: 15, nlat: 384, nlon: 320)
Coordinates:
  * year           (year) int64 0 1 2 3 4 5 6 7 8 ... 53 54 55 56 57 58 59 60 61
  * z_t_150m       (z_t_150m) float32 500.0 1.5e+03 ... 1.35e+04 1.45e+04
    TLAT           (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.19 72.19
    TLONG          (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
Dimensions without coordinates: nlat, nlon
Data variables:
    phytoC_summer  (year, z_t_150m, nlat, nlon) float32 0.0 0.0 0.0 ... 0.0 0.0
    phytoC_spring  (year, z_t_150m, nlat, nlon) float32 0.0 0.0 0.0 ... 0.0 0.0
    phytoC_fall    (year, z_t_150m, nlat, nlon) float32 0.0 0.0 0.0 ... 0.0 0.0
    phytoC_winter  (year, z_t_150m, nlat, nlon) float32 0.0 0.0 0.0 ... 0.0 0.0

In [15]:
%%time
phytoC_summer_bm = np.nansum(phytoC_summer, axis=(0,2)) * 10/150 #integrate over depth #mmol m-3
phytoC_summer_bm = phytoC_summer_bm * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

phytoC_spring_bm = np.nansum(phytoC_spring, axis=(0,2)) * 10/150 #integrate over depth #mmol m-3
phytoC_spring_bm = phytoC_spring_bm * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

phytoC_fall_bm = np.nansum(phytoC_fall, axis=(0,2)) * 10/150 #integrate over depth #mmol m-3
phytoC_fall_bm = phytoC_fall_bm * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

phytoC_winter_bm = np.nansum(phytoC_winter, axis=(0,2)) * 10/150 #integrate over depth #mmol m-3
phytoC_winter_bm = phytoC_winter_bm * 12.011 *1000 # ug C m-3 --> ugC m^{-3}

CPU times: user 23.3 s, sys: 1min 11s, total: 1min 35s
Wall time: 6min 6s


In [24]:
ds.lat

<xarray.DataArray 'lat' (lat: 180)>
array([-89.5, -88.5, -87.5, -86.5, -85.5, -84.5, -83.5, -82.5, -81.5, -80.5,
       -79.5, -78.5, -77.5, -76.5, -75.5, -74.5, -73.5, -72.5, -71.5, -70.5,
       -69.5, -68.5, -67.5, -66.5, -65.5, -64.5, -63.5, -62.5, -61.5, -60.5,
       -59.5, -58.5, -57.5, -56.5, -55.5, -54.5, -53.5, -52.5, -51.5, -50.5,
       -49.5, -48.5, -47.5, -46.5, -45.5, -44.5, -43.5, -42.5, -41.5, -40.5,
       -39.5, -38.5, -37.5, -36.5, -35.5, -34.5, -33.5, -32.5, -31.5, -30.5,
       -29.5, -28.5, -27.5, -26.5, -25.5, -24.5, -23.5, -22.5, -21.5, -20.5,
       -19.5, -18.5, -17.5, -16.5, -15.5, -14.5, -13.5, -12.5, -11.5, -10.5,
        -9.5,  -8.5,  -7.5,  -6.5,  -5.5,  -4.5,  -3.5,  -2.5,  -1.5,  -0.5,
         0.5,   1.5,   2.5,   3.5,   4.5,   5.5,   6.5,   7.5,   8.5,   9.5,
        10.5,  11.5,  12.5,  13.5,  14.5,  15.5,  16.5,  17.5,  18.5,  19.5,
        20.5,  21.5,  22.5,  23.5,  24.5,  25.5,  26.5,  27.5,  28.5,  29.5,
        30.5,  31.5,  32.5,  33.5,  34.5,  35.5,  36.5,  37.5,  38.5,  39.5,
        40.5,  41.5,  42.5,  43.5,  44.5,  45.5,  46.5,  47.5,  48.5,  49.5,
        50.5,  51.5,  52.5,  53.5,  54.5,  55.5,  56.5,  57.5,  58.5,  59.5,
        60.5,  61.5,  62.5,  63.5,  64.5,  65.5,  66.5,  67.5,  68.5,  69.5,
        70.5,  71.5,  72.5,  73.5,  74.5,  75.5,  76.5,  77.5,  78.5,  79.5,
        80.5,  81.5,  82.5,  83.5,  84.5,  85.5,  86.5,  87.5,  88.5,  89.5])
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y

In [16]:
phytoC_summer_bm.shape

(62, 384, 320)

In [17]:
phytoC_summer_bm[phytoC_summer_bm<0] ==np.nan
phytoC_spring_bm[phytoC_spring_bm<0] ==np.nan
phytoC_fall_bm[phytoC_fall_bm<0] ==np.nan
phytoC_winter_bm[phytoC_winter_bm<0] ==np.nan

array([], dtype=bool)

In [26]:
phyto_bmss = xr.DataArray(phytoC_summer_bm, coords={'year':ds_fall_avg.year, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['year', 'nlat', 'nlon'])
phyto_bmss = phyto_bmss.to_dataset(name='phytoC_summer_bm')

phyto_bmss['phytoC_spring_bm'] = xr.DataArray(phytoC_spring_bm, coords={'year':ds_fall_avg.year, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['year', 'nlat', 'nlon'])

phyto_bmss['phytoC_fall_bm'] = xr.DataArray(phytoC_fall_bm, coords={'year':ds_fall_avg.year, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['year', 'nlat', 'nlon'])

phyto_bmss['phytoC_winter_bm'] = xr.DataArray(phytoC_winter_bm, coords={'year':ds_fall_avg.year, 'TLAT':ds_fall_avg.TLAT, 'TLONG':ds_fall_avg.TLONG},
                      dims=['year', 'nlat', 'nlon'])

In [39]:
phyto_bmss.to_netcdf('/glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr.nc');

	NC4_create: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr.nc cmode 0x1000 parameters (nil)
	HDF5 error messages turned on.
			nc4_create_file: path /glade/u/home/gabyn/scratch/SPECTRA/nc_files/phyto_bmss_62yr.nc mode 0x1000
			nc4_grp_list_add: name / 
		nc_inq_format: ncid 0x700000
		NC4_inq_format_extended: ncid 0x700000
		nc_inq_typeids: ncid 0x700000
		NC4_inq: ncid 0x700000
		NC4_inq: ncid 0x700000
		nc_inq_grps: ncid 0x700000
		NC4_def_dim: ncid 0x700000 name year len 62
		NC4_def_dim: ncid 0x700000 name nlat len 384
		NC4_def_dim: ncid 0x700000 name nlon len 320
		NC4_def_var: name year type 10 ndims 1
		NC4_inq_unlimdims: ncid 0x700000
		NC4_inq_var_all: ncid 0x700000 varid 0
		NC4_inq_var_all: ncid 0x700000 varid 0
		NC4_inq_var_all: ncid 0x700000 varid 0
		NC4_inq_var_all: ncid 0x700000 varid 0
		NC4_inq_dim: ncid 0x700000 dimid 0
		NC4_inq_dim: ncid 0x700000 dimid 0
		NC4_inq_var_all: ncid 0x700000 varid 0
		NC4_inq_var_all: ncid 0x700000 varid 0
		NC4_i

## Bring in Grid information

In [40]:
## Bring in Grid information -- /glade/u/home/gabyn/scratch/SPECTRA/GNG595_2ndcycle_1990-2009_clim/
grid_inds = sorted(glob('/glade/u/home/gabyn/scratch/SPECTRA/GNG595_2ndcycle_1990-2009_clim/regrid/regrid_SPECTRA_TAREA.*.nc', recursive=True))
ds= xr.open_mfdataset(grid_inds,concat_dim='time', combine='nested') # Full 62 years of second cycle

In [41]:
ds.TAREA[0,:,:]

<xarray.DataArray 'TAREA' (lat: 180, lon: 360)>
dask.array<getitem, shape=(180, 360), dtype=float64, chunksize=(180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float64 -180.0 -179.0 -178.0 -177.0 ... 177.0 178.0 179.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Attributes:
    long_name:  area of T cells
    units:      centimeter^2

## Import photoC_TOT

## High vs low temperature

In [42]:
temp_inds = sorted(glob('/glade/scratch/gabyn/SPECTRA/GNG595_monthly_yearly_1948_2009/regrid/regrid_SPECTRA_TEMP.*.nc', recursive=True))
ds_01= xr.open_mfdataset(temp_inds[1::],concat_dim='time', combine='nested') # Full 62 years of second cycle

In [43]:
%%time
temp_ds_inter = np.empty([62,12,180,360])
temp_ds_inter[:] = np.nan
x = np.array(range(0,744,12)) ## this loops from 0 to 2016, and selects every 12th number. 
for i in range(62):
    temp_ds_inter[i,:,:,:] = ds_01.TEMP[x[i]:x[i]+12,0,:,:]

CPU times: user 9.74 s, sys: 8.13 s, total: 17.9 s
Wall time: 30 s


In [44]:
inter = xr.DataArray(temp_ds_inter, coords={'year':ds_fall_avg.year, 'time':ds.time, 'lat':ds.lat, 'lon':ds.lon},
                      dims=['year', 'time', 'lat', 'lon'])
inter = inter.to_dataset(name='temp_ds_inter')
temp_arctic = esmlab.statistics.weighted_sum(inter.temp_ds_inter[:,:,149:180,:], weights=ds.TAREA[0,149:180,:], dim=['lat', 'lon'])

In [45]:
#temp_arctic = np.nanmean(temp_ds_inter[:,:,149:180,:], axis=(2,3))
temp_arctic_anom = temp_arctic-np.nanmean(temp_arctic,axis=0)
err_ao = np.nanstd(temp_arctic_anom,axis=(0,1))

# Selecting the warm and cold years as less or more than 1/3 of standard deviation 
cold_years_arctic = np.where(np.nanmean(temp_arctic_anom,axis=1) < -(err_ao/2))
warm_years_arctic = np.where(np.nanmean(temp_arctic_anom,axis=1) > (err_ao/2))

In [46]:
warm_years_arctic

(array([ 0,  2,  5,  6, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]),)

In [47]:
cold_years_summer = phytoC_summer_bm[cold_years_arctic,:,:] # 1/2 standard deviation 
warm_years_summer = phytoC_summer_bm[warm_years_arctic,:,:] # 1/2 standard deviation 

cold_years_spring = phytoC_spring_bm[cold_years_arctic,:,:] # 1/2 standard deviation 
warm_years_spring = phytoC_spring_bm[warm_years_arctic,:,:] # 1/2 standard deviation 

cold_years_fall = phytoC_fall_bm[cold_years_arctic,:,:] # 1/2 standard deviation 
warm_years_fall = phytoC_fall_bm[warm_years_arctic,:,:] # 1/2 standard deviation 

cold_years_winter = phytoC_winter_bm[cold_years_arctic,:,:] # 1/2 standard deviation 
warm_years_winter = phytoC_winter_bm[warm_years_arctic,:,:] # 1/2 standard deviation 

## High vs low Ice Fraction

In [48]:
temp_inds = sorted(glob('/glade/scratch/gabyn/SPECTRA/GNG595_monthly_yearly_1948_2009/regrid/regrid_SPECTRA_ECOSYS_IFRAC*.nc', recursive=True))
ds_02= xr.open_mfdataset(temp_inds[1::],concat_dim='time', combine='nested') # Full 62 years of second cycle

	NC4_close: ncid 0x650000
			nc4_close_hdf5_file: h5->path /glade/scratch/gabyn/phyto_bmss_62yr.nc abort 0
			nc4_rec_grp_HDF5_del: grp->name /
			closing HDF5 dataset 360287970189639689
			closing HDF5 dataset 360287970189639692
			closing HDF5 dataset 360287970189639695
			closing HDF5 dataset 360287970189639698
			closing HDF5 dataset 360287970189639702
			closing HDF5 dataset 360287970189639706
			closing HDF5 dataset 360287970189639710
			nc4_close_netcdf4_file: h5->path /glade/scratch/gabyn/phyto_bmss_62yr.nc abort 0
			nc4_rec_grp_del: grp->name /
			att_free: name _FillValue 
			att_free: name long_name 
			att_free: name units 
			att_free: name _FillValue 
			att_free: name long_name 
			att_free: name units 
			att_free: name _FillValue 
			att_free: name coordinates 
			att_free: name _FillValue 
			att_free: name coordinates 
			att_free: name _FillValue 
			att_free: name coordinates 
			att_free: name _FillValue 
			att_free: name coordinates 


In [49]:
ECOSYS_IFRAC_ds_inter = np.empty([62,12,180,360])
ECOSYS_IFRAC_ds_inter[:] = np.nan
x = np.array(range(0,744,12)) ## this loops from 0 to 2016, and selects every 12th number. 
for i in range(62):
    ECOSYS_IFRAC_ds_inter[i,:,:,:] = ds_02.ECOSYS_IFRAC[x[i]:x[i]+12,:,:]

In [50]:
inter = xr.DataArray(ECOSYS_IFRAC_ds_inter, coords={'year':ds_fall_avg.year, 'time':ds.time, 'lat':ds.lat, 'lon':ds.lon},
                      dims=['year', 'time', 'lat', 'lon'])
inter = inter.to_dataset(name='ECOSYS_IFRAC_ds_inter')
ice_arctic = esmlab.statistics.weighted_sum(inter.ECOSYS_IFRAC_ds_inter[:,:,149:180,:], weights=ds.TAREA[0,149:180,:], dim=['lat', 'lon'])

In [51]:
#ice_arctic = np.nanmean(ECOSYS_IFRAC_ds_inter[:,:,149:180,:], axis=(2,3))
ice_arctic_anom = ice_arctic-np.nanmean(ice_arctic,axis=0)
err_ao = np.nanstd(ice_arctic_anom,axis=(0,1))

# Selecting the warm and cold years as less or more than 1/3 of standard deviation 
low_ice_years_arctic = np.where(np.nanmean(ice_arctic_anom,axis=1) < -(err_ao/2))
high_ice_years_arctic = np.where(np.nanmean(ice_arctic_anom,axis=1) > (err_ao/2))

In [52]:
low_ice_years_summer = phytoC_summer_bm[low_ice_years_arctic,:,:] # 1/2 standard deviation 
high_ice_years_summer = phytoC_summer_bm[high_ice_years_arctic,:,:] # 1/2 standard deviation 

low_ice_years_spring = phytoC_spring_bm[low_ice_years_arctic,:,:] # 1/2 standard deviation 
high_ice_years_spring = phytoC_spring_bm[high_ice_years_arctic,:,:] # 1/2 standard deviation 

low_ice_years_fall = phytoC_fall_bm[low_ice_years_arctic,:,:] # 1/2 standard deviation 
high_ice_years_fall = phytoC_fall_bm[high_ice_years_arctic,:,:] # 1/2 standard deviation 

low_ice_years_winter = phytoC_winter_bm[low_ice_years_arctic,:,:] # 1/2 standard deviation 
high_ice_years_winter = phytoC_winter_bm[high_ice_years_arctic,:,:] # 1/2 standard deviation 

## High vs low NO$_3$

In [53]:
NO3_inds = sorted(glob('/glade/scratch/gabyn/SPECTRA/GNG595_monthly_yearly_1948_2009/regrid/regrid_SPECTRA_NO3*.nc', recursive=True))
ds_03= xr.open_mfdataset(NO3_inds[1::],concat_dim='time', combine='nested') # Full 62 years of second cycle

In [54]:
NO3_ds_inter = np.empty([62,12,180,360])
NO3_ds_inter[:] = np.nan
x = np.array(range(0,744,12)) ## this loops from 0 to 2016, and selects every 12th number. 
for i in range(62):
    NO3_ds_inter[i,:,:,:] = ds_03.NO3[x[i]:x[i]+12,0,:,:]

In [55]:
inter = xr.DataArray(NO3_ds_inter, coords={'year':ds_fall_avg.year, 'time':ds.time, 'lat':ds.lat, 'lon':ds.lon},
                      dims=['year', 'time', 'lat', 'lon'])
inter = inter.to_dataset(name='NO3_ds_inter')
no3_arctic = esmlab.statistics.weighted_sum(inter.NO3_ds_inter[:,:,149:180,:], weights=ds.TAREA[0,149:180,:], dim=['lat', 'lon'])

In [56]:
#no3_arctic = np.nanmean(NO3_ds_inter[:,:,149:180,:], axis=(2,3))
no3_arctic_anom = no3_arctic-np.nanmean(no3_arctic,axis=0)
err_ao = np.nanstd(no3_arctic_anom,axis=(0,1))

# Selecting the warm and cold years as less or more than 1/3 of standard deviation 
low_no3_years_arctic = np.where(np.nanmean(no3_arctic_anom,axis=1) < -(err_ao/2))
high_no3_years_arctic = np.where(np.nanmean(no3_arctic_anom,axis=1) > (err_ao/2))

In [57]:
low_no3_years_summer = phytoC_summer_bm[low_no3_years_arctic,:,:] # 1/2 standard deviation 
high_no3_years_summer = phytoC_summer_bm[high_no3_years_arctic,:,:] # 1/2 standard deviation 

low_no3_years_spring = phytoC_spring_bm[low_no3_years_arctic,:,:] # 1/2 standard deviation 
high_no3_years_spring = phytoC_spring_bm[high_no3_years_arctic,:,:] # 1/2 standard deviation 

low_no3_years_fall = phytoC_fall_bm[low_no3_years_arctic,:,:] # 1/2 standard deviation 
high_no3_years_fall = phytoC_fall_bm[high_no3_years_arctic,:,:] # 1/2 standard deviation 

low_no3_years_winter = phytoC_winter_bm[low_no3_years_arctic,:,:] # 1/2 standard deviation 
high_no3_years_winter = phytoC_winter_bm[high_no3_years_arctic,:,:] # 1/2 standard deviation 

In [60]:
low_no3_years_arctic

(array([ 0,  1,  2,  3,  4,  5,  6,  7, 12, 13, 50, 53, 54, 55, 56, 58]),)

## Save all of this on an array, so you don't have to go through this every time! 

In [65]:
np.save('/glade/u/home/gabyn/scratch/SPECTRA/GNG595_npy_files/low_no3_years_arctic.npy',low_no3_years_arctic)
np.save('/glade/u/home/gabyn/scratch/SPECTRA/GNG595_npy_files/high_no3_years_arctic.npy',high_no3_years_arctic)
np.save('/glade/u/home/gabyn/scratch/SPECTRA/GNG595_npy_files/low_ice_years_arctic.npy',low_ice_years_arctic)
np.save('/glade/u/home/gabyn/scratch/SPECTRA/GNG595_npy_files/high_ice_years_arctic.npy',high_ice_years_arctic)
np.save('/glade/u/home/gabyn/scratch/SPECTRA/GNG595_npy_files/low_temp_years_arctic.npy',cold_years_arctic)
np.save('/glade/u/home/gabyn/scratch/SPECTRA/GNG595_npy_files/high_temp_years_arctic.npy',warm_years_arctic)